In [12]:
import compute_rhino3d
import compute_rhino3d.Grasshopper as gh
from compute_rhino3d.Util import DecodeToCommonObject, DecodeToPoint3d, DecodeToLine, DecodeToBoundingBox, DecodeToVector3d
from rhino3dm import (File3dm,CommonObject,ObjectAttributes,
Point,Point2d,Point2f,Point3d,Point4d,PointCloud,Vector2d,Vector3d,Vector3f,
Line,LineCurve,Mesh,Plane,
Curve,Polyline,PolylineCurve,Curve,Arc,BezierCurve,Circle,Ellipse,NurbsCurve,
Surface,
BoundingBox,Cone,Cylinder,Extrusion,Sphere,Brep,
Transform)
from json import loads,dumps,JSONDecodeError

from pydantic import BaseModel


In [13]:

def encode(value):
    if type(value).__module__.startswith('rhino3dm'):
        return dumps(value.Encode())
    else:
        return value

def prepareParameter(parameter):
    return encode(parameter)

def convertToTrees(parameters):
    trees = []
    if type(parameters) is not dict:
        return "Parameters have to be a dictionary"
    
    for kw in parameters:
        branches = parameters[kw]
        tree = gh.DataTree(kw)
        #check if branches is just item
        if type(branches) != list:
            tree.Append([0], [prepareParameter(branches)])
        else:
            #check if branches is list
            if type(branches[0]) != list:
                tree.Append([0], [prepareParameter(item) for item in branches])
            else:
                #branches must be a "real" tree with many branches
                for i,branch in enumerate(branches):
                    tree.Append([i], [prepareParameter(item) for item in branch])
        trees.append(tree)
    return trees

def parseSingleValue(valueReply):
    try:
        if 'type' in valueReply:
            valueType =  valueReply['type']
            data =  loads(valueReply['data'])
            if valueType.endswith('Point3d'):
                return Point3d(data['X'],data['Y'],data['Z'])
            else:
                return CommonObject.Decode(data)
        return valueReply
    except:
        return valueReply

def parseSingleResults(valuesReply):
    results = {}
    log = []
    for valueReply in valuesReply['values']:
        tree = valueReply['InnerTree']
        if len(tree)!=1:
            log.append("Tree did not have exactly one 1 branch" +valueReply)
            continue
        value =list(tree.values())[0]
        if len(value)!=1:
            log.append("Branch did not have exactly one 1 value")
        results[valueReply['ParamName']]=parseSingleValue(value[0])
    return (results,log)

"""Simplified call with dictionaries as input."""
def callGrasshopper(path,parameters = {}, computeUrl = "http://localhost:6500/", computeAuthToken= ""):
    trees = convertToTrees(parameters)
    try:
        compute_rhino3d.Util.url = computeUrl
        compute_rhino3d.Util.authToken = computeAuthToken
        return gh.EvaluateDefinition(path, trees)
    except JSONDecodeError as e:
            raise ValueError("Probably the path was wrong. Make sure to give an absolute path otherwise the call will most likely fail. This is the log\n" + str(e))
    except Exception as e: return "The call to compute went wrong. " + str(e)

In [14]:
decodeDict= {
    "Rhino.Geometry.Point3d": DecodeToPoint3d,
    "Rhino.Geometry.Vector3d": DecodeToVector3d,
    # 'Rhino.Geometry.Box': DecodeToCommonObject,
    'Rhino.Geometry.PolylineCurve': DecodeToCommonObject
}
def decode(item):
    data = loads(item['data'])
    type = item['type']
    if not type in decodeDict:
        raise ValueError(f'The item type {type} is currently not supported.')
    return decodeDict[type](data)

def addObjectToModel(item,model):
    if type(item) in [Point, Point2d, Point2f, Point2d, Point3d, Point4d]:
        model.Objects.AddPoint(item)
    elif type(item) in [PointCloud]:
        model.Objects.AddPointCloud(item)
    elif type(item) in [Line]:
        model.Objects.AddLine(item)
    elif type(item) in [Polyline]:
        model.Objects.AddPolyline(item)
    elif type(item) in [Arc]:
        model.Objects.AddArc(item)
    elif type(item) in [Circle]:
        model.Objects.AddCircle(item)
    elif type(item) in [Ellipse]:
        model.Objects.AddEllipse(item)
    elif type(item) in [Sphere]:
        model.Objects.AddSphere(item)
    elif type(item) in [Curve, PolylineCurve]:
        model.Objects.AddCurve(item)
    elif type(item) in [Surface]:
        model.Objects.AddSurface(item)
    elif type(item) in [Extrusion]:
        model.Objects.AddExtrusion(item)
    elif type(item) in [Mesh]:
        model.Objects.AddMesh(item)
    elif type(item) in [Brep]:
        model.Objects.AddBrep(item)
    elif type(item) in [Surface]:
        model.Objects.AddSurface(item)
    else:
        model.Objects.Add(item) 


def parseFileFromOutput(grasshopperReply, outputName):
    model = File3dm()
    valuesDictionary = { valueReply['ParamName']:valueReply['InnerTree'] for valueReply in grasshopperReply['values']}
    if not outputName in valuesDictionary:
        raise ValueError(f'Output name {outputName} was not in grasshopper reply.')
    representationLayerIndex = model.Layers.AddLayer(outputName,(0,0,0,255))
    representationObjectAttributes = ObjectAttributes()
    for branch in valuesDictionary[outputName].values():
        print (branch)
        for item in branch:
            try:
                #print(item)
                decodedObject = decode(item)
                #print(decodedObject)
                addObjectToModel(decodedObject,model)
            except Exception as e:
                print(e)
    return model
   
    # representationObjectAttributes.LayerIndex = representationLayerIndex
    # model.Objects.Add(rectangleWithMiter1,representationObjectAttributes)
    # afterLayerIndex = model.Layers.AddLayer("Related",(0,0,0,255))
    # afterObjectAttributes = ObjectAttributes()
    # afterObjectAttributes.LayerIndex = afterLayerIndex
    # connection.placeRelatedModule(model,afterObjectAttributes)
    # model.Write

In [15]:
response = callGrasshopper(r"C:\Git\Studium\PhD\semio\tests\elements\RectangleWithMiter.gh")
response


{'absolutetolerance': 0.0,
 'angletolerance': 0.0,
 'modelunits': 'Millimeters',
 'algo': '',
 'pointer': 'md5_57C503BFBCA1262A6EB6444090BB2F81',
 'cachesolve': False,
 'recursionlevel': 0,
 'values': [{'ParamName': 'ATTRACTIONPOINT',
   'InnerTree': {'{0}': [{'type': 'Rhino.Geometry.Point3d',
      'data': '{"X":200.0,"Y":110.0,"Z":0.0}'}]}},
  {'ParamName': 'REPRESENTATION',
   'InnerTree': {'{0;0;0;0}': [{'type': 'Rhino.Geometry.Mesh',
      'data': '{"version":10000,"archive3dm":60,"opennurbs":-1905753079,"data":"+n8CALIEAAAAAAAA+/8CABQAAAAAAAAA5NTXTkfp0xG/5QAQgwEi8C25G1z8/wIAegQAAAAAAAA4MAAAACQAAAAAAAAAAAAAAAIAAAAA9O8/AAAAAAAAAABVVVVVVUnlPwAAAAAAAAAAAAAAAABgg0AAAAAAAAAAAAAAAAAAYINAAAAAAAAAAAAAAAAAAAAAAAAAgD8AAIA/AACAPwAAgL8AAIC/AACAvwAAgD8AAIA/AACAPwAAgL8AAIC/AACAvwAAgD8AAIA/AACAvwAAgL//////AQCAAECVAAAAAAAAABUAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAALUMc6+I2Gj8AAAAAAAAAAAAAAAAAABhAAAAAAAAAAACHROdKGFfWPwAAAAAAAPA/h0TnShhX1j+HROdKGFe2PwAAAAACAAAAAAAAAAAAAAAAAAEAgABAFQAAAAAAAAABAAAAAgAAA

In [16]:
model = parseFileFromOutput(response,'REPRESENTATION')

[{'type': 'Rhino.Geometry.Mesh', 'data': '{"version":10000,"archive3dm":60,"opennurbs":-1905753079,"data":"+n8CALIEAAAAAAAA+/8CABQAAAAAAAAA5NTXTkfp0xG/5QAQgwEi8C25G1z8/wIAegQAAAAAAAA4MAAAACQAAAAAAAAAAAAAAAIAAAAA9O8/AAAAAAAAAABVVVVVVUnlPwAAAAAAAAAAAAAAAABgg0AAAAAAAAAAAAAAAAAAYINAAAAAAAAAAAAAAAAAAAAAAAAAgD8AAIA/AACAPwAAgL8AAIC/AACAvwAAgD8AAIA/AACAPwAAgL8AAIC/AACAvwAAgD8AAIA/AACAvwAAgL//////AQCAAECVAAAAAAAAABUAAAAAAAAAAAEAAAAAAAAAAAAAAAAAAAAAAAAALUMc6+I2Gj8AAAAAAAAAAAAAAAAAABhAAAAAAAAAAACHROdKGFfWPwAAAAAAAPA/h0TnShhX1j+HROdKGFe2PwAAAAACAAAAAAAAAAAAAAAAAAEAgABAFQAAAAAAAAABAAAAAgAAAAQAAAACAAAAANmNOQnXEQBhAAAAAAEAAAAGAAQEBgUBAQcGBAQGBwUFBwMFBQIHBAQOCAwMDg0JCQ8ODAwODw0NDwsNDQoPDAwWEBQUFhURERcWFBQWFxUVFxMVFRIXFBQfGBwcHx0ZGR4fHBwfHh0dHhsdHRoeHBwkICcnJCUhISYkJyckJiUlJiMlJSImJycvKCwsLy4pKS0vLCwvLS4uLSsuLiotLCxAAgAAz7Nt4QEAgABAXQAAAAAAAAB42qWRUQ4AEAxDd7ce03FcTCRIdZ0fksaMvFIRrUcWlni95jl2/9SQHnKtnMsDt5h7GMo0e5Yrb7L3e3kXnDK3bjKCz4zPJR+Xl7CZUfp8ZM95+X8YMV6GEdxAMEdAAgAAeFH2jAEAgABALAAAAAAAAAB42mNgAIGG/

In [17]:

byteModel


b'3D Geometry File Format       70\x01\x00\x00\x00~\x00\x00\x00\x00\x00\x00\x00 Runtime: ON::RuntimeEnvironment::Windows 3DM I/O processor: OpenNURBS toolkit version 2386329284 (compiled on Mar 23 2022)\n\x1a\x00\x14\x00\x00\x10\x0b\x01\x00\x00\x00\x00\x00\x00&\x00\x00\xa0\xc4~<\x8e\x00\x00\x00\x00\'\x80\x00 v\x00\x00\x00\x00\x00\x00\x007\x00\x00\x00C\x00:\x00\\\x00U\x00s\x00e\x00r\x00s\x00\\\x00U\x00e\x00l\x00i\x00\\\x00A\x00p\x00p\x00D\x00a\x00t\x00a\x00\\\x00L\x00o\x00c\x00a\x00l\x00\\\x00T\x00e\x00m\x00p\x00\\\x00t\x00m\x00p\x00p\x001\x00z\x00v\x003\x00n\x00v\x00u\x00\\\x00m\x00o\x00d\x00e\x00l\x00.\x003\x00d\x00m\x00\x00\x00\xd2\x90\xe0m!\x80\x00 e\x00\x00\x00\x00\x00\x00\x00\x10\x05\x00\x00\x00U\x00e\x00l\x00i\x00\x00\x00%\x00\x00\x002\x00\x00\x00\x12\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x06\x00\x00\x00\r\x00\x00\x00\x05\x00\x00\x00U\x00e\x00l\x00i\x00\x00\x00%\x00\x00\x002\x00\x00\x00\x12\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00{\x00\x00\x00\x06\x00\x00\x0

In [18]:
file = File3dm.FromByteArray(byteModel)
file.Write('after.3dm')


True

In [ ]:
Any()